In [1]:
pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-m9v9g9hr
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-m9v9g9hr
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0-cp36-none-any.whl size=80383 sha256=a8c85adb35bc23ce661c364c7b91572bf9df2db813f28a8f26784affe59f563e
  Stored in directory: /tmp/pip-ephem-wheel-cache-59fzv_fr/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [2]:
import pandas as pd
from sklearn import preprocessing 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [3]:
df = pd.read_excel("Train_dataset.xlsx")
df.tail()

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,cardiological pressure,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
10709,22691,Bengaluru,Male,Mr,ashish,NO,2.0,Business,Car,8,2,Diabetes,20,14,<400,Normal,134.0,67.0,24.0,209.0,61.0,56.0,35,7,4000000.0,300000,0.0,41.981760
10710,22692,Bengaluru,Male,Mr,mohd mujahid ansari,YES,0.0,Manufacturing,Car,8,2,None,42,4,<400,Normal,387.0,102.0,134.0,211.0,65.0,47.0,35,7,1500000.0,700000,NaN,47.148663
10711,22693,Bengaluru,Male,Mr,munnakumar,YES,0.0,Clerk,Public,8,2,Diabetes,59,3,<100,Stage-02,177.0,111.0,151.0,296.0,87.0,37.0,16,5,3000000.0,2100000,2.0,47.708533
10712,22694,Bengaluru,Male,Mr,prakash,YES,1.0,Legal,Walk,8,2,Coronary Heart Disease,49,6,<300,Elevated,352.0,140.0,110.0,253.0,56.0,70.0,28,5,2900000.0,700000,1.0,53.803565
10713,22695,Bengaluru,Male,Mr,mehboob mehandi,NO,2.0,Business,Car,8,2,Diabetes,17,7,<400,Normal,181.0,65.0,134.0,201.0,82.0,60.0,4,3,2100000.0,1400000,0.0,68.342400


In [4]:
df.isna().sum()

people_ID                    0
Region                       0
Gender                       0
Designation                  0
Name                        20
Married                      0
Children                   311
Occupation                 747
Mode_transport               3
cases/1M                     0
Deaths/1M                    0
comorbidity                226
Age                          0
Coma score                   0
Pulmonary score              0
cardiological pressure      97
Diuresis                   716
Platelets                  924
HBB                         16
d-dimer                   1114
Heart rate                1114
HDL cholesterol             35
Charlson Index               0
Blood Glucose                0
Insurance                 1090
salary                       0
FT/month                   723
Infect_Prob                  0
dtype: int64

In [0]:
df = df.dropna()  #dropping all rows with missing values. - data cleaning

In [6]:
df.tail()

,people_ID,Region,Gender,Designation,Name,Married,Children,Occupation,Mode_transport,cases/1M,Deaths/1M,comorbidity,Age,Coma score,Pulmonary score,cardiological pressure,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Infect_Prob
10708,22690,Bengaluru,Male,Mr,manoj kumar,YES,1.0,Farmer,Public,8,2,Diabetes,35,11,<200,Stage-01,397.0,78.0,150.0,337.0,69.0,44.0,31,3,3300000.0,1400000,2.0,53.741750
10709,22691,Bengaluru,Male,Mr,ashish,NO,2.0,Business,Car,8,2,Diabetes,20,14,<400,Normal,134.0,67.0,24.0,209.0,61.0,56.0,35,7,4000000.0,300000,0.0,41.981760
10711,22693,Bengaluru,Male,Mr,munnakumar,YES,0.0,Clerk,Public,8,2,Diabetes,59,3,<100,Stage-02,177.0,111.0,151.0,296.0,87.0,37.0,16,5,3000000.0,2100000,2.0,47.708533
10712,22694,Bengaluru,Male,Mr,prakash,YES,1.0,Legal,Walk,8,2,Coronary Heart Disease,49,6,<300,Elevated,352.0,140.0,110.0,253.0,56.0,70.0,28,5,2900000.0,700000,1.0,53.803565
10713,22695,Bengaluru,Male,Mr,mehboob mehandi,NO,2.0,Business,Car,8,2,Diabetes,17,7,<400,Normal,181.0,65.0,134.0,201.0,82.0,60.0,4,3,2100000.0,1400000,0.0,68.342400


In [0]:
output =np.array(df.loc[:,"Infect_Prob"])

In [0]:
for i in range(len(output)):
  output[i] = output[i]/100

In [9]:
output

array([0.4913501 , 0.73224   , 0.48779225, ..., 0.47708533, 0.53803565,
       0.683424  ])

In [0]:
df = df.drop('Infect_Prob', axis=1)
df = df.drop('people_ID',axis=1)

In [0]:
#df1 = pd.get_dummies(df['Region'],prefix='Region')
#df = pd.concat([df,df1],axis =1)
catg = ['Mode_transport', 'comorbidity', 'cardiological pressure','Pulmonary score'] #catagorical data
for i in range(4):
  df1 = pd.get_dummies(df[catg[i]],prefix=catg[i])
  df = pd.concat([df,df1],axis =1)
  df = df.drop(catg[i],axis=1)
df = df.drop(['Designation','Name','Region','Occupation'],axis = 1)  #redundant data- we remove this to make the model more effecient 
label_encoder = preprocessing.LabelEncoder()
df['Married'] = label_encoder.fit_transform(df['Married'])  #label encoding
df['Gender'] = label_encoder.fit_transform(df['Gender'])

In [12]:
stats = df.describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
Gender,6777.0,4.980080e-01,5.000329e-01,0.0,0.0,0.0,1.0,1.0
Married,6777.0,7.761546e-01,4.168504e-01,0.0,1.0,1.0,1.0,1.0
Children,6777.0,1.001180e+00,7.071058e-01,0.0,1.0,1.0,2.0,2.0
cases/1M,6777.0,3.299985e+00,2.739398e+00,1.0,2.0,2.0,4.0,8.0
Deaths/1M,6777.0,5.233879e-01,8.600869e-01,0.0,0.0,0.0,1.0,2.0
Age,6777.0,4.277438e+01,1.573889e+01,16.0,28.0,46.0,55.0,68.0
Coma score,6777.0,9.021691e+00,3.704153e+00,3.0,6.0,9.0,12.0,15.0
Diuresis,6777.0,2.784809e+02,9.820198e+01,110.0,194.0,276.0,365.0,450.0
Platelets,6777.0,8.734071e+01,4.174577e+01,15.0,52.0,88.0,124.0,160.0
HBB,6777.0,1.153360e+02,5.555070e+01,20.0,67.0,116.0,164.0,210.0


In [0]:
def build_model():   # we change and tune the hyper parameters of this model to give least loss
  model = keras.models.Sequential()
  model.add(layers.Dense(128,input_shape=(33,)))
  #model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(64,activation = 'relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(32,activation = 'relu'))
  model.add(layers.Dense(1, activation = 'sigmoid'))
  model.compile(optimizer='adam', loss= 'mse')
  return model

In [14]:
model = build_model()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               4352      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dens

In [0]:
#ex = df[:10]
#ex_r = model.predict(ex)
#ex_r

In [16]:
def norm(x):
  return ((x - stats['mean']) / stats['std'])
norm_data = norm(df)
norm_data.tail()

,Gender,Married,Children,cases/1M,Deaths/1M,Age,Coma score,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Mode_transport_Car,Mode_transport_Public,Mode_transport_Walk,comorbidity_Coronary Heart Disease,comorbidity_Diabetes,comorbidity_Hypertension,comorbidity_None,cardiological pressure_Elevated,cardiological pressure_Normal,cardiological pressure_Stage-01,cardiological pressure_Stage-02,Pulmonary score_<100,Pulmonary score_<200,Pulmonary score_<300,Pulmonary score_<400
10708,1.003918,0.536992,-0.001669,1.715711,1.716817,-0.493960,0.534079,1.206891,-0.223752,0.624007,1.410790,-0.386655,-0.844352,0.922937,-1.448767,0.473856,0.150549,1.218381,-0.695582,1.428312,-0.725684,-0.579295,1.738250,-0.577251,-0.577478,-0.582021,-0.582249,1.747220,-0.572708,-0.572708,1.747220,-0.582021,-0.582249
10709,1.003918,-1.861950,1.412546,1.715711,1.716817,-1.447014,1.343980,-1.471263,-0.487252,-1.644191,-1.521573,-0.925077,0.309758,1.260225,1.427546,0.979780,-1.655820,-1.245650,1.437432,-0.700024,-0.725684,-0.579295,1.738250,-0.577251,-0.577478,-0.582021,1.717226,-0.572253,-0.572708,-0.572708,-0.572253,-0.582021,1.717226
10711,1.003918,0.536992,-1.415885,1.715711,1.716817,1.030925,-1.625659,-1.033389,0.566747,0.642008,0.471518,0.824793,-1.517582,-0.341891,-0.010611,0.257031,1.300056,1.218381,-0.695582,1.428312,-0.725684,-0.579295,1.738250,-0.577251,-0.577478,-0.582021,-0.582249,-0.572253,1.745834,1.745834,-0.572253,-0.582021,-0.582249
10712,1.003918,0.536992,-0.001669,1.715711,1.716817,0.395556,-0.815758,0.748652,1.261428,-0.096056,-0.513573,-1.261591,1.656219,0.669971,-0.010611,0.184756,-0.998958,-0.013635,-0.695582,-0.700024,1.377807,1.725980,-0.575206,-0.577251,-0.577478,1.717896,-0.582249,-0.572253,-0.572708,-0.572708,-0.572253,1.717896,-0.582249
10713,1.003918,-1.861950,1.412546,1.715711,1.716817,-1.637624,-0.545790,-0.992657,-0.535161,0.335982,-1.704846,0.488280,0.694461,-1.353754,-1.448767,-0.393444,0.150549,-1.245650,1.437432,-0.700024,-0.725684,-0.579295,1.738250,-0.577251,-0.577478,-0.582021,1.717226,-0.572253,-0.572708,-0.572708,-0.572253,-0.582021,1.717226


In [17]:
EPOCHS = 1000
history = model.fit(norm_data,output,epochs=EPOCHS, validation_split = 0.2, verbose=0,callbacks=[tfdocs.modeling.EpochDots()])
# we give a validation set to avoid over-fitting


Epoch: 0, acc:0.0000,  loss:0.6914,  val_acc:0.0000,  val_loss:0.6852,  
....................................................................................................
Epoch: 100, acc:0.0000,  loss:0.6712,  val_acc:0.0000,  val_loss:0.6913,  
....................................................................................................
Epoch: 200, acc:0.0000,  loss:0.6699,  val_acc:0.0000,  val_loss:0.6927,  
....................................................................................................
Epoch: 300, acc:0.0000,  loss:0.6696,  val_acc:0.0000,  val_loss:0.6912,  
....................................................................................................
Epoch: 400, acc:0.0000,  loss:0.6685,  val_acc:0.0000,  val_loss:0.6917,  
....................................................................................................
Epoch: 500, acc:0.0000,  loss:0.6682,  val_acc:0.0000,  val_loss:0.6896,  
..............................................

Accuracy stays zero because we are trying to REGRESS the probability of infection and not classify people.

In [0]:
#Data set cleaning for Test data
dft = pd.read_excel("Test_dataset.xlsx")
pid =np.array(dft.loc[:,"people_ID"])
pid =pd.DataFrame(pid)
dft = dft.drop('people_ID',axis=1)

In [0]:
catg = ['Mode_transport', 'comorbidity', 'cardiological pressure','Pulmonary score']
for i in range(4):
  df1 = pd.get_dummies(dft[catg[i]],prefix=catg[i])
  dft = pd.concat([dft,df1],axis =1)
  dft = dft.drop(catg[i],axis=1)
dft = dft.drop(['Region','Designation','Name','Occupation'],axis = 1)

In [0]:
label_encoder = preprocessing.LabelEncoder()
dft['Married'] = label_encoder.fit_transform(dft['Married'])
dft['Gender'] = label_encoder.fit_transform(dft['Gender'])

In [35]:
norm_test = norm(dft)
norm_test.tail()
# we use the statistics of the training data set for the test data set too.

,Gender,Married,Children,cases/1M,Deaths/1M,Age,Coma score,Diuresis,Platelets,HBB,d-dimer,Heart rate,HDL cholesterol,Charlson Index,Blood Glucose,Insurance,salary,FT/month,Mode_transport_Car,Mode_transport_Public,Mode_transport_Walk,comorbidity_Coronary Heart Disease,comorbidity_Diabetes,comorbidity_Hypertension,comorbidity_None,cardiological pressure_Elevated,cardiological pressure_Normal,cardiological pressure_Stage-01,cardiological pressure_Stage-02,Pulmonary score_<100,Pulmonary score_<200,Pulmonary score_<300,Pulmonary score_<400
14493,1.003918,0.536992,-1.415885,2.810842,1.716817,0.776778,-1.085725,1.410553,-1.708933,-1.428173,1.616972,-0.588564,-0.748176,1.175903,-1.448767,1.124330,-0.834743,1.218381,-0.695582,-0.700024,1.377807,1.725980,-0.575206,-0.577251,-0.577478,-0.582021,1.717226,-0.572253,-0.572708,-0.572708,-0.572253,-0.582021,1.717226
14494,-0.995950,0.536992,-1.415885,2.810842,1.716817,0.077872,-1.355692,-0.809361,-1.565206,-1.554184,0.059154,1.295912,-1.517582,0.248362,0.708468,1.341155,0.807410,1.218381,1.437432,-0.700024,-0.725684,-0.579295,-0.575206,-0.577251,1.731412,1.717896,-0.582249,-0.572253,-0.572708,-0.572708,-0.572253,1.717896,-0.582249
14495,1.003918,0.536992,-1.415885,2.810842,1.716817,0.204946,-0.815758,-1.196319,-1.732887,-1.572185,0.425699,-0.117445,0.309758,-1.185110,0.708468,1.196605,-0.834743,-0.013635,-0.695582,1.428312,-0.725684,-0.579295,-0.575206,-0.577251,1.731412,1.717896,-0.582249,-0.572253,-0.572708,-0.572708,-0.572253,1.717896,-0.582249
14496,-0.995950,0.536992,-1.415885,2.810842,1.716817,-0.557497,-1.085725,-1.063939,-1.349615,-1.644191,-1.109210,0.555583,0.694461,1.007259,1.427546,-0.176619,0.807410,1.218381,1.437432,-0.700024,-0.725684,1.725980,-0.575206,-0.577251,-0.577478,-0.582021,-0.582249,1.747220,-0.572708,-0.572708,1.747220,-0.582021,-0.582249
14497,-0.995950,0.536992,-1.415885,2.810842,1.716817,-1.002255,-1.355692,-1.552727,-1.708933,-0.942130,0.150790,0.353674,0.598285,-0.426213,-0.729689,-0.899368,-0.506312,-1.245650,-0.695582,1.428312,-0.725684,-0.579295,1.738250,-0.577251,-0.577478,-0.582021,-0.582249,1.747220,-0.572708,-0.572708,1.747220,-0.582021,-0.582249


In [0]:
predicted = model.predict(norm_test)

In [37]:
type(predicted)

numpy.ndarray

In [38]:
print(predicted)
for i in range (len(predicted)):
  predicted[i] = predicted[i] * 100
predicted= pd.DataFrame(predicted)
predicted = pd.concat([pid,predicted],axis=1)
predicted.to_csv("results.csv")

[[0.50628626]
 [0.5164933 ]
 [0.53277236]
 ...
 [0.45850566]
 [0.4606093 ]
 [0.4609647 ]]
